In [23]:
!pip install syft

In [0]:
import torch 

In [0]:
def get_parallel_db(db, remove_index):
  return torch.cat((db[0:remove_index], db[remove_index+1:]))

In [0]:
def get_parallel_dbs(db):
  
  parallel_dbs = list()
  
  for i in range(len(db)):
    pdb = get_parallel_db(db, i)
    parallel_dbs.append(pdb)
    
  return parallel_dbs

In [0]:
 def create_db_and_parallels(num_entries):
    
    db = torch.rand(num_entries)>0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [0]:
def sensitivity(query, n_entries=1000):
  db, pdbs = create_db_and_parallels(n_entries)
  
  full_db_result=query(db)
  
  max_distance = 0

  for pdb in pdbs:
    pdb_result = query(pdb)
  
    db_distance=torch.abs(pdb_result - full_db_result)
  
    if(db_distance > max_distance):
      max_distance = db_distance
    
  return max_distance

In [0]:
def query(db, threshold=5):
  return (db.sum()> threshold).float()

In [23]:
db, pdbs = create_db_and_parallels(10)
db.sum()

tensor(5)

In [24]:
for i in range(10):
  sens_f = sensitivity(query, n_entries=10)
  print(sens_f)

0
0
0
0
0
0
0
0
tensor(1.)
0


In [0]:
db, _ = create_db_and_parallels(100)

In [0]:
pdb = get_parallel_db(db, remove_index=10)

In [27]:
db[10]

tensor(0, dtype=torch.uint8)

In [28]:
# differencing attack using sum query
sum(db) - sum(pdb)

tensor(0, dtype=torch.uint8)

In [29]:
# differencing attack using mean query

(sum(db).float() / len(db)) - (sum(pdb.float()/ len(pdb)))

tensor(-0.0045)

In [30]:
# differencing attack using threshold query

(sum(db).float() > 49 ) - (sum(pdb.float()> 49 ))

tensor(0, dtype=torch.uint8)

In [0]:
epsilon = 0.5

In [0]:
import numpy as np

In [0]:
db , pdbs = create_db_and_parallels(100)

In [34]:
db

tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
        1, 0, 0, 1], dtype=torch.uint8)

In [0]:
def sum_query(db):
  return db.sum()

In [0]:
def laplacian_mechanism(db, query, sensitivity):
  beta = sensitivity / epsilon
  noise = torch.tensor(np.random.laplace(0,beta,1))
  

  return query(db)+ noise

In [52]:
sum_query(db)

tensor(40)

In [55]:
laplacian_mechanism(db, sum_query, 1)

tensor([41.0078], dtype=torch.float64)

In [0]:
def mean_query(db):
  return torch.mean(db.float())

In [62]:
laplacian_mechanism(db, mean_query, 1/100)

tensor([0.4119], dtype=torch.float64)